# Question 1: Permutation tests

In class, we briefly mentioned permutation testing as an alternative to bootstrapping a two-sample test.

Recall that with bootstrapping, you shift the data around to comply with the null hypothesis (often that the data sets have identical means). Then you repeatedly resample the transformed data and compute your statistic on the resampled data to generate a null distribution, which tells you how common it would be to see any given value of the statistic by chance if the data sets were actually not distinct.

With permutation testing, we use the null hypothesis that the various data groups (i.e. "control" and "experimental", but this works for multi-group data as well) are actually all just drawn from a single big, indistinguishable group. If this were true, then it shouldn't matter which data points are assigned to which group -- because the groups are all identical anyway. Therefore, we can produce a null distribution by randomly shuffling (or "permuting") the groups to see how common it is to get any given difference between the groups by chance alone.

Effectively, this is asking how often the observed configuration of the group labels (or one that yields similarly extreme values for some statistic) might come about simply due to chance alone. 

For this, we will use the same statistics and p-value functions as in the bootstrapping exercies. All that needs to be done to implement a permutation test is to write code that will reshuffle group labels -- or, equivalently, randomly assign each data point to one of the data groups.

The simplest way to approach this problem is to assume that we have $i$ different groups, with sizes $n_i$, for a total of $k = \sum_i n_i$ data points. If we concatenate all these data points together into a single list that is $k$ elements long, then the first $n_1$ data points will belong to group 1, the next $n_2$ points will belong to group 2, and so forth, up to the last $n_i$ points which belong to group $i$. 

For example, imagine  we have two groups of data: `control = [1, 4, 3, 5, 3]` (5 elements) and `experimental = [3, 5, 6, 5]` (4 elements). Put together, the concatenated data set is `all_data = [1, 4, 3, 5, 3, 3, 5, 6, 5]`, where `control = all_data[0:5]` and `experimental = all_data[5:9]`.

If we shuffled the `all_data` list around (not by resampling with replacement! Just re-ordering the elements...), then our new "control" group would be `shuffled_data[0:5]` and our new "experimental" group would be `shuffled_data[5:9]`. Conveniently, the function `numpy.random.permutation` will return a permuted copy of its input array.

Try the code out below:

In [ ]:
import numpy

group1 = numpy.array([1, 1, 1, 1, 1, 1])
group2 = numpy.array([2, 2, 2, 2])
group3 = numpy.array([3, 3, 3, 3, 3, 3, 3, 3])
groups = [group1, group2, group3]
all_data = numpy.concatenate(groups)

print('orig:', all_data)
print()

print('perm 1:', numpy.random.permutation(all_data))
print('perm 2:', numpy.random.permutation(all_data))
print('perm 3:', numpy.random.permutation(all_data))
print()

lengths = [len(group) for group in groups]
print('lengths:', lengths)

## Question 1.0
Write a function `split_data` that will split up an array or list into a set of sub-lists with specified lengths. We will use the function to split up a concatenated `all_data` array such as above back into groups of the requested sizes. 

Specifically, your function should be: `split_data(all_data, lengths)`, where `all_data` is a list or array, and `lengths` is a list of lengths of the required sub-lists. Note that `sum(lengths)` will always be equal to `len(all_data)` -- you could even enforce this via an assert.

Here is an example of how the function should work:
```python
group1 = numpy.array([1])
group2 = numpy.array([2, 3])
group3 = numpy.array([4, 5, 6])
groups = [group1, group2, group3]

lengths = [len(group) for group in groups]
# i.e. lengths = [1, 2, 3]
all_data = numpy.concatenate(groups)

split_groups = split_data(all_data, lengths)
# split_groups should be [numpy.array([1]), numpy.array([2, 3]), numpy.array([4, 5, 6])
```

You can either do this with a for loop and a counter to record the index where the current sub-group starts in the `all_data` array, or (for a brainteaser / style points) you could look up the documentation for `numpy.cumsum` and `numpy.split` and figure out how to reduce the operation to one line.

In [ ]:
import numpy

# YOUR ANSWER HERE

group1 = numpy.array([1])
group2 = numpy.array([2, 3])
group3 = numpy.array([4, 5, 6])
groups = [group1, group2, group3]

lengths = [len(group) for group in groups]
all_data = numpy.concatenate(groups)

split_groups = split_data(all_data, lengths)
print(split_groups)

In [ ]:
new_lengths = [2, 4, 6, 8, 10]
all_data = numpy.arange(sum(new_lengths))
split_groups = split_data(all_data, new_lengths)
assert len(split_groups) == len(new_lengths)
assert numpy.all(numpy.concatenate(split_groups) == all_data)
for length, group in zip(new_lengths, split_groups):
    assert len(group) == length

## Question 1.1
Using your `split_data` function and `numpy.random.permutation`, write a function `permutation_p(data_sets, statistic, n_permutations=10000)`, where:
 - `data_sets` is a list of two or more data sets
 - `statistic` is a function to calculate a statistic from two or more data sets
 - `n_permutations` is the number of permutations to examine
 
The function should return `stat`, `permuted_stats`, `p`, where:
 - `stat` is the statistic calculated on the actual data sets
 - `permuted_stats` is the list of statistics computed on the permuted data
 - `p` is the two-tailed p-value.

Several helper functions are provided below. Note that with a permutation test, the null hypothesis is always that a given statistic has zero difference between the groups.

In [ ]:
def two_tail_p_value(actual_stat, resampled_stats, null_hyp_stat=0):
    resampled_stats = numpy.asarray(resampled_stats)
    actual_diff = abs(actual_stat - null_hyp_stat)
    diff = numpy.abs(resampled_stats - null_hyp_stat)
    count_extreme = numpy.count_nonzero(diff >= actual_diff)
    return count_extreme / len(resampled_stats)

# YOUR ANSWER HERE

# load blood pressure data again...
aa_bp = []
gg_bp = []
file = open('bloodpressure3.txt')
header = file.readline()
for line in file:
    bp, genotype = line.strip('\n').split('\t')
    bp = float(bp)
    if genotype == 'AA':
        aa_bp.append(bp)
    elif genotype == 'GG':
        gg_bp.append(bp)
    else:
        print('unknown genotype!', genotype)
file.close()
aa_bp = numpy.array(aa_bp)
gg_bp = numpy.array(gg_bp)


def mean_difference(data1, data2):
    return numpy.mean(data1) - numpy.mean(data2)

mean_diff, perm_diffs, perm_p = permutation_p([aa_bp, gg_bp], mean_difference)

def resample(data_sets, statistic, n_resamples):
    data_sets = [numpy.asarray(data_set) for data_set in data_sets]
    stats = []
    for i in range(n_resamples):
        resampled_data_sets = [numpy.random.choice(data, size=len(data), replace=True) for data in data_sets]
        stats.append(statistic(*resampled_data_sets))
    return stats

def bootstrap_means_different(data_sets, statistic, n_resamples=10000):
    data_sets = [numpy.asarray(data_set) for data_set in data_sets]
    actual_stat = statistic(*data_sets)
    shifted = [data_set - numpy.mean(data_set) for data_set in data_sets]
    resampled_stats = resample(shifted, statistic, n_resamples)
    p_val = two_tail_p_value(actual_stat, resampled_stats)
    return actual_stat, resampled_stats, p_val

mean_diff, boot_diffs, boot_p = bootstrap_means_different([aa_bp, gg_bp], mean_difference)

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = [12, 4]

plt.hist(perm_diffs, bins='auto', label='permutation null')
plt.hist(boot_diffs, bins='auto', label='bootstrap null', alpha=0.6)
plt.axvline(mean_diff, color='blue')
plt.legend()
print('permutation p:', perm_p)
print('bootstrap p:', boot_p)

In [ ]:
assert len(perm_diffs) == 10000
assert -0.1 < numpy.mean(perm_diffs) < 0.1
assert 0.007 < perm_p < 0.014

Very interesting! The permutation test yields  consistently smaller p-values, at least in this case. Of course, this is only a good thing if these small p-values aren't false positives! We'll test that in the question about statistical power.

The neat thing here is that the permutation code will work for almost any statistic. There is no requirement to transform the data, which has to be done slightly differently for each different statistic of interest. (We saw this in the last problem set: we had to modify the basic bootstrap p-value code depending on if we wanted to subtract the mean or the median of each data set, or divide by the standard deviation, or whatnot...)

## Question 1.2
Modify the above code to produce functions `median_difference` and `bootstrap_medians_different`. Then use your `permutation_p` and `bootstrap_medians_different` functions to perform the analysis as above, except on the difference in medians instead of means.

Store the results of `permutation_p` in variables `median_diff, perm_diffs, perm_p`, and of `bootstrap_medians_different` in `median_diff, boot_diffs, boot_p`.

The code below also plots the GG and AA histograms. Below we will compare them to understand how and why the p-values are different for the mean and median case!

In [ ]:
# YOUR ANSWER HERE

plt.hist(perm_diffs, bins='auto', label='permutation null')
plt.hist(boot_diffs, bins='auto', label='bootstrap null', alpha=0.6)
plt.axvline(median_diff, color='blue')
plt.legend()

plt.figure()
plt.hist(gg_bp, bins='auto', label='GG')
plt.hist(aa_bp, bins='auto', alpha=0.6, label='AA')

plt.axvline(numpy.mean(gg_bp), color='yellow', label='GG mean')
plt.axvline(numpy.median(gg_bp), color='orange', label='GG median')
plt.axvline(numpy.mean(aa_bp), color='cyan', label='AA mean')
plt.axvline(numpy.median(aa_bp), color='blue', label='AA median')

plt.legend()
print('permutation p:', perm_p)
print('bootstrap p:', boot_p)

In [ ]:
assert 0.32 < perm_p < 0.36
assert 0.42 < boot_p < 0.46

As you can see from the GG and AA histograms, there are a few outliers at the low tail of the AA dataset, giving it a slightly different shape than the GG histogram. (This is largely what makes the difference in the means is much larger than the difference in medians between these two data sets.)

Does this have anything to do with why the shape of the null distribution is so different for the bootstrap and permutation tests? In particular, why is there such a large and asymmetric mass of probability around $\textrm{median}(AA)-\textrm{median}(GG)=-5$ for the bootstrapped null? We'll find out below.

## Question 1.3
Take 10,000 resamples of `gg_bp`, calculate the median of each, and store the result in a list `gg_medians`. Do the same for `aa_bp` and store it as `aa_medians`. 

Next, calculate the "standard error of the median" for each of these datasets, and store it as `sterr_gg` and `sterr_aa`. Remember that the standard error of a statistic is just the standard deviation of the distribution of where that statistic might appear in repeated samplings. Which of course we approximate by looking at where the statistic appears in repeated *resamplings* (such as were just calculated).

The code will also plot histograms of these resamples. Is there anything interesting about the shape of the `aa_medians` histogram compared to `gg_medians`?

In [ ]:
# YOUR ANSWER HERE

plt.hist(gg_medians, bins='auto', label='GG resampled medians')
plt.hist(aa_medians, bins='auto', alpha=0.6, label='AA resampled medians')
plt.axvline(numpy.median(gg_bp), color='orange', label='GG median')
plt.axvline(numpy.median(aa_bp), color='blue', label='AA median')
plt.legend()
print('standard error of the AA median', sterr_aa)
print('standard error of the GG median', sterr_gg)

In [ ]:
assert numpy.median(gg_bp) == numpy.median(gg_medians)
assert numpy.median(aa_bp) == numpy.median(aa_medians)
assert len(gg_medians) == len(aa_medians) == 10000
# the average resampled median is very close to the actual median for the GG data
assert abs(numpy.median(gg_bp) - numpy.median(gg_medians)) < 0.1
# the average resampled median is actually a little smaller than the actual median for the AA data
assert 1.0 < numpy.median(aa_bp) - numpy.mean(aa_medians) < 1.2
assert 2.8 < sterr_aa < 3.0
assert 1.0 < sterr_gg < 1.2

As you can see, the resampling process will occasionally draw enough of the low-valued outliers from the AA dataset to actually drag the median of an AA resample quite far from the true median of the AA data. (This  also causes a  much larger standard error for that dataset than the GG data.)

In contrast, the permutation test always and only uses the original dataset. Thus, there is no chance of drawing an "AA" dataset with a large fraction of outliers like that. This leads to the more orderly, symmetric null distribution for the permutation, as seen in the histogram in question 1.2.

So, which approach is right? Well, it depends! Is the slightly heavy tail in the AA dataset a sampling artifact that should be ignored? Or a true feature of the population with the AA genotype, which should be paid attention to? Statistics can't help us here!

For the last two questions, we'll apply the permutation test to the other scenarios we did bootstrapping for in last homework: the ratio of the standard deviations, and an ANOVA analysis.

## Question 1.4
Let's test whether the AA and GG data have different standard deviations. Last time this required heavily modifying the bootstrapping code to ask about whether a ratio was larger or smaller than 1. Let's be a little more clever this time, and use a standard trick from statistics: the question of whether a ratio is larger or smaller than 1 is the same is the question of whether the logarithm of that ratio is positive or negative. (That is, as usual, one can transform a question concerning multiplicative scaling into one concerning additive scaling by using the logarithm.) 

Write a function `log_std_ratio(data1, data2)`, using `numpy.log` and `numpy.std` that will return the log of the ratio of the standard deviations of `data1` and `data2`. Use this to conduct a permutation test of the AA and GG standard deviations (make sure you order the data sets such that the ratio calculated is std(AA)/std(GG)...). Compare those results to the bootstrap test of the same (using the provided code, which is a simplification of the code you were asked to write for the last homework).

Store the results of `permutation_p` in variables `log_ratio, perm_logs, perm_p`, and of `bootstrap_std_different` in `log_ratio, boot_logs, boot_p`.

In [ ]:
def log_std_ratio(data1, data2):
    # YOUR ANSWER HERE

def unit_stds(data1, data2):
    data1 = numpy.asarray(data1)
    data2 = numpy.asarray(data2)
    mean1 = numpy.mean(data1)
    mean2 = numpy.mean(data2)
    unit1 = (data1 - mean1) / numpy.std(data1) + mean1
    unit2 = (data2 - mean1) / numpy.std(data2) + mean2
    return unit1, unit2

def bootstrap_std_different(data1, data2, n_resamples=10000):
    actual_log = log_std_ratio(data1, data2)
    unit1, unit2 = unit_stds(data1, data2)
    resampled_logs = resample([unit1, unit2], log_std_ratio, n_resamples)
    p_val = two_tail_p_value(actual_log, resampled_logs)
    return actual_log, resampled_logs, p_val


# Now calculate p-values from a permutation and bootstrap test for the log std ratio.
# YOUR ANSWER HERE


plt.hist(perm_logs, bins='auto', label='permutation null')
plt.hist(boot_logs, bins='auto', label='bootstrap null', alpha=0.6)
plt.axvline(log_ratio, color='blue')
plt.legend()
print('std ratio:', numpy.std(aa_bp) / numpy.std(gg_bp))
print('permutation p:', perm_p)
print('bootstrap p:', boot_p)

In [ ]:
assert log_std_ratio(aa_bp, gg_bp) == log_ratio
assert abs(numpy.exp(log_ratio) - numpy.std(aa_bp) / numpy.std(gg_bp)) < 0.0000001
assert 0.028 < perm_p < 0.042
assert 0.018 < boot_p < 0.032

Here, we see very similar results. (Now the bootstrap test is giving miniscually lower p-values than the permutation test, but things are very comparable.)

## Question 1.5
Finally, let's use the permutation test to do a four-sample ANOVA, again using the examples from the last homework.

Store the results of `permutation_p` in variables `f_val, perm_fs, perm_p`, and of `bootstrap_anova` in `f_val, boot_fs, boot_p`.

In [ ]:
dose0 = numpy.array([
        0.44353122,  0.01012966,  0.28873548,  0.41508564,  0.63920005,
        0.38840745,  0.56994441,  0.39461312,  0.63531554,  0.2243476 ,
        0.0146331 ,  0.34027282,  0.11461404,  0.3706856 ,  0.03022053,
        0.26903881,  0.40752708,  0.57226634,  0.56036688,  0.45346642,
        0.24393661,  0.11314218,  0.37424453,  0.02731236,  0.36170049,
        0.28624436,  0.27673137,  0.02059751,  0.33590967,  0.34159704])

dose1 = numpy.array([
        0.40447368,  0.04894949,  0.43691414,  0.36156942,  0.55082948,
        0.45234243,  0.08263355,  0.53735037,  0.19556559,  0.14684213,
        0.12670269,  0.09968791,  0.09903378,  0.23750902,  0.30392521,
        0.27720465,  0.64437095,  0.62903639,  0.62678483,  0.41837535,
        0.02564389,  0.34572511,  0.48980329,  0.67060822,  0.40915237,
        0.67581135,  0.63069843,  0.1503978 ,  0.12660994,  0.25990044])

dose2 = numpy.array([
        0.2023481 ,  0.17263792,  0.24061181,  0.23310464,  0.75607685,
        0.67992754,  0.1684292 ,  0.72103724,  0.50828973,  0.19148194,
        0.76176207,  0.31807931,  0.04664898,  0.22524644,  0.83347876,
        0.58708986,  0.40128552,  0.33489851,  0.16898191,  0.0937998 ,
        0.82937534,  0.69083288,  0.14723692,  0.51264047,  0.46377266,
        0.55422885,  0.49082234,  0.80979359,  0.43772453,  0.35936454])

dose3 = numpy.array([
        0.56307295,  0.260053  ,  0.65498897,  0.4074656 ,  0.38936101,
        0.10776682,  0.46990373,  0.39733009,  0.04181612,  0.15412012,
        0.44600949,  0.56160325,  0.09421104,  0.29194078,  0.69317676,
        0.22814752,  0.17328897,  0.40468055,  0.69985375,  0.39791401,
        0.55387151,  0.68740186,  0.52072545,  0.65457568,  0.19796182,
        0.58060724,  0.6489602 ,  0.2342047 ,  0.42521541,  0.62100328])

plt.scatter([0]*30, dose0)
plt.scatter([1]*30, dose1)
plt.scatter([2]*30, dose2)
plt.scatter([3]*30, dose3)

def F_stat(*data_sets):
    data_sets = [numpy.asarray(data) for data in data_sets]
    grand_mean = numpy.mean(numpy.concatenate(data_sets))
    # note: the sum of squares from a dataset to its mean is
    # (per the definition of variance) the variance times the number of data points
    ss_within = sum([numpy.var(data) * len(data) for data in data_sets])
    ss_between = sum([len(data) * (numpy.mean(data) - grand_mean)**2 for data in data_sets])
    return ss_between / ss_within

def bootstrap_anova(data_sets, n_resamples=10000):
    actual_stat = F_stat(*data_sets)
    # set all means to zero instead of grand mean. Works the same
    shifted = [data - numpy.mean(data) for data in data_sets]
    resampled_stats = resample(shifted, F_stat, n_resamples)
    p_val = two_tail_p_value(actual_stat, resampled_stats)
    return actual_stat, resampled_stats, p_val

# Now calculate p-values from a permutation and bootstrap test of the F statistic.
# YOUR ANSWER HERE

plt.figure()
plt.hist(perm_fs, bins='auto', label='permutation null')
plt.hist(boot_fs, bins='auto', label='bootstrap null', alpha=0.6)
plt.axvline(f_val, color='blue')
plt.legend()
print('permutation p:', perm_p)
print('bootstrap p:', boot_p)

In [ ]:
assert 0.1 < perm_p < 0.13
assert 0.1 < boot_p < 0.13

As you can see, the permutation test is very general for multi-sample tests, and works almost identically to bootstrap hypothesis testing. Nevertheless, it's important to understand how each works in order to understand the cases in which the tests differ a little bit. Both are useful and commonly-employed statistical methods, and so it's good to have both in your tool-belt.